## Imports

In [22]:
from importlib import reload
reload(model)

<module 'model' from 'C:\\Users\\franc\\Desktop\\EPFL\\Deep Learning\\Miniprojects\\Proj_338157_338681_311699\\Miniproject2\\model.py'>

In [3]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath('model.py'))))
import model

In [4]:
import torch
import torch.nn as nn

In [20]:
def test_linear():
    torch.manual_seed(0)

    torch.set_grad_enabled(True)
    linear_torch = nn.Linear(100, 1000)
    initial_weight = linear_torch.weight.clone()
    initial_bias = linear_torch.bias.clone()
    # Batch size x input dimension
    x_test = torch.zeros((5, 100)).normal_().requires_grad_()
    output = linear_torch(x_test)
    gradwrtoutput = torch.zeros(output.shape).normal_()
    output.backward(gradwrtoutput)

    # Testing dimensions, initialization
    torch.set_grad_enabled(False)
    linear_model = model.Linear(100, 1000)
    assert linear_model.weight.shape == initial_weight.shape
    assert linear_model.bias.shape == initial_bias.shape
    assert torch.allclose(linear_model.weight.mean(), initial_weight.mean(), atol=1e-3, rtol=1e-2)
    assert torch.allclose(linear_model.weight.std(), initial_weight.std(), atol=1e-3, rtol=1e-2)

    linear_model.weight = initial_weight
    linear_model.bias = initial_bias
    linear_model.zero_grad()
    output_model = linear_model(x_test)
    gradwrtinput = linear_model.backward(gradwrtoutput)

    assert output_model.shape == output.shape
    assert torch.allclose(output_model, output)
    assert torch.allclose(gradwrtinput, x_test.grad)
    assert torch.allclose(linear_torch.weight.grad, linear_model.weight.grad)
    assert torch.allclose(linear_torch.bias.grad, linear_model.bias.grad)
    
    print('Linear layer ok')

In [23]:
test_linear()

Linear layer ok


In [26]:
def test_conv():
    torch.manual_seed(0)

    torch.set_grad_enabled(True)
    conv_torch = nn.Conv2d(32, 64, stride=2, padding=2, dilation=2, kernel_size=5)
    initial_weight = conv_torch.weight.clone()
    initial_bias = conv_torch.bias.clone()
    # Batch size x input dimension
    x_test = torch.zeros((5, 32, 50, 50)).normal_().requires_grad_()
    output = conv_torch(x_test)
    gradwrtoutput = torch.zeros(output.shape).normal_()
    output.backward(gradwrtoutput)

    # Testing dimensions, initialization
    torch.set_grad_enabled(False)
    conv_model = model.Conv2d(32, 64, stride=2, padding=2, dilation=2, kernel_size=5)
    assert conv_model.weight.shape == initial_weight.shape
    assert conv_model.bias.shape == initial_bias.shape
    assert torch.allclose(conv_model.weight.mean(), initial_weight.mean(), atol=1e-2, rtol=1e-2)
    assert torch.allclose(conv_model.weight.var(), initial_weight.var(), atol=1e-2, rtol=1e-2)

    conv_model.weight = initial_weight
    conv_model.bias = initial_bias
    conv_model.zero_grad()
    output_model = conv_model(x_test)
    gradwrtinput = conv_model.backward(gradwrtoutput)

    assert output_model.shape == output.shape
    assert torch.allclose(output_model, output, atol=1e-2)
    assert torch.allclose(gradwrtinput, x_test.grad, atol=1e-2)
    assert torch.allclose(conv_torch.weight.grad, conv_model.weight.grad, atol=1e-2)
    assert torch.allclose(conv_torch.bias.grad, conv_model.bias.grad, atol=1e-2)
    
    print('Convolutional layer ok')

In [28]:
test_conv()

Convolutional layer ok


In [29]:
def test_relu():
    torch.manual_seed(0)

    torch.set_grad_enabled(True)
    relu_torch = nn.ReLU()
    # Batch size x input dimension
    x_test = torch.zeros((5, 100)).normal_().requires_grad_()
    output = relu_torch(x_test)
    gradwrtoutput = torch.zeros(output.shape).normal_()
    output.backward(gradwrtoutput)

    # Testing dimensions, initialization
    torch.set_grad_enabled(False)
    relu_model = model.ReLU()
    output_model = relu_model(x_test)
    gradwrtinput = relu_model.backward(gradwrtoutput)

    assert output_model.shape == output.shape
    assert torch.allclose(output_model, output)
    assert torch.allclose(gradwrtinput, x_test.grad)
    
    print('ReLU layer ok')

In [30]:
test_relu()

ReLU layer ok


In [31]:
def test_sigmoid():
    torch.manual_seed(0)

    torch.set_grad_enabled(True)
    sigmoid_torch = nn.Sigmoid()
    # Batch size x input dimension
    x_test = torch.zeros((5, 100)).normal_().requires_grad_()
    output = sigmoid_torch(x_test)
    gradwrtoutput = torch.zeros(output.shape).normal_()
    output.backward(gradwrtoutput)

    # Testing dimensions, initialization
    torch.set_grad_enabled(False)
    sigmoid_model = model.Sigmoid()
    output_model = sigmoid_model(x_test)
    gradwrtinput = sigmoid_model.backward(gradwrtoutput)

    assert output_model.shape == output.shape
    assert torch.allclose(output_model, output)
    assert torch.allclose(gradwrtinput, x_test.grad)
    
    print('Sigmoid layer ok')

In [32]:
test_sigmoid()

Sigmoid layer ok


In [33]:
def test_mse():
    torch.manual_seed(0)

    torch.set_grad_enabled(True)
    mse_torch = nn.MSELoss()
    # Batch size x input dimension
    target = torch.zeros((5, 100)).normal_()
    x_test = torch.zeros((5, 100)).normal_().requires_grad_()
    output = mse_torch(x_test, target)
    gradwrtoutput = torch.zeros(output.shape).normal_()
    output.backward(gradwrtoutput)

    # Testing dimensions, initialization
    torch.set_grad_enabled(False)
    mse_model = model.MSE()
    output_model = mse_model(x_test, target)
    gradwrtinput = mse_model.backward(gradwrtoutput)

    assert output_model.shape == output.shape
    assert torch.allclose(output_model, output)
    assert torch.allclose(gradwrtinput, x_test.grad, atol=1e-2, rtol=1e-2)
    
    print('MSE layer ok')

In [34]:
test_mse()

MSE layer ok


In [37]:
def test_nearest_upsampling():
    torch.manual_seed(0)

    torch.set_grad_enabled(True)
    upsample_torch = nn.Upsample(scale_factor=3)
    # Batch size x input dimension
    x_test = torch.zeros((5, 32, 8, 8)).normal_().requires_grad_()
    output = upsample_torch(x_test)
    gradwrtoutput = torch.zeros(output.shape).normal_()
    output.backward(gradwrtoutput)

    # Testing dimensions, initialization
    torch.set_grad_enabled(False)
    upsample_model = model.NearestUpsampling(scale_factor=3)
    output_model = upsample_model(x_test)
    gradwrtinput = upsample_model.backward(gradwrtoutput)

    assert output_model.shape == output.shape
    assert torch.allclose(output_model, output)
    assert torch.allclose(gradwrtinput, x_test.grad, atol=1e-2, rtol=1e-2)
    
    print('Nearest upsample layer ok')

In [38]:
test_nearest_upsampling()

Nearest upsample layer ok


In [39]:
def test_all():
    test_linear()
    test_conv()
    test_relu()
    test_sigmoid()
    test_mse()
    test_nearest_upsampling()
    print('All tests ok')

In [40]:
test_all()

Linear layer ok
Convolutional layer ok
ReLU layer ok
Sigmoid layer ok
MSE layer ok
Nearest upsample layer ok
All tests ok
